In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install emoji

In [1]:
import os
import random
import numpy as np
import glob
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import json
import os
from datasets import Dataset
from transformers import *
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import re
import gc
import emoji
import zipfile

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# X = pd.read_parquet('/content/drive/MyDrive/datasocio/clean_twibot2.pq')
X_meta = pd.read_parquet("/content/drive/MyDrive/datasocio/X_meta2.pq")

In [4]:
X_meta.head()

,profile_followers_count,profile_friends_count,is_mt,is_reply,text_sat,has_ellipsis,unk_chars_ratio,cashtags_count,hashtags_count,links_count,mentions_count,emojis_count
0,15349596,692,0,0,0.583333,True,1.0,0,0,0,0,2
1,15349596,692,0,0,0.658333,False,1.0,0,0,2,1,0
2,15349596,692,0,0,1.266667,False,1.0,0,1,2,1,0
3,15349596,692,0,0,0.695833,False,1.0,0,2,2,1,0
4,15349596,692,0,0,0.075000,False,1.0,0,1,0,0,0


In [5]:
X_meta.shape

(1398465, 12)

In [6]:
batch_size = 64
max_length = 128
start_lr = 2e-4
min_lr = 1e-8
epochs = 10

In [7]:
def compute_metrics(pred):
    labels = pred.label_ids
    #map to 01
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    roc = roc_auc_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [8]:
model_ckpt = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_ckpt)

https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp8c3he91e


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpekcj0dfk


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpt9_u3z4g


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
creating metadata file for /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.0

In [9]:
input_ids = np.load('/content/drive/MyDrive/datasocio/input_ids2.npy', allow_pickle = True)
attention_masks= np.load('/content/drive/MyDrive/datasocio/attention_masks2.npy', allow_pickle = True)
target = np.load('/content/drive/MyDrive/datasocio/target2.npy',allow_pickle = True)
# target = np.array(pd.get_dummies(X['labels']))

In [10]:
AUTOTUNE = tf.data.AUTOTUNE

# Note that some tokenizers also returns 'token_id'. Modify this function accordingly. 
@tf.function
def parse_data(inputs, target):
    inputs_ids = inputs['input_ids']
    attention_mask = inputs['attention']
    meta = inputs['meta']
    target = tf.cast(target, tf.int32)
    
    return {'input_ids': input_ids,
            'attention_mask': attention_mask,
            'meta': meta}, target

In [11]:
config = AutoConfig.from_pretrained(model_ckpt) 
backbone = TFDistilBertModel.from_pretrained(model_ckpt, config = config)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.18.0",
  "vocab_size": 30522
}

https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 not found in cache or force_download set to True, downloading to /root/.cache/huggingface/tr

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
creating metadata file for /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the

In [12]:
backbone.summary()

Model: "tf_distil_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
Total params: 66,362,880
Trainable params: 66,362,880
Non-trainable params: 0
_________________________________________________________________


In [13]:
def build_model():
    
    input_ids = tf.keras.Input(shape=(128,),dtype='int32', name = 'input_ids')
    attention_masks = tf.keras.Input(shape=(128,),dtype='int32', name = 'attention')
    meta = tf.keras.Input(shape=(X_meta.shape[1],),dtype='int32', name = 'meta') #just followers for now, don't know how to deal with rest
    
    output = backbone(input_ids,attention_masks)[0]
    
    output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(output)
    output = tf.keras.layers.GlobalMaxPool1D()(output)
    
    output = tf.keras.layers.Dense(16,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    meta_output = tf.keras.layers.Dense(16,activation='relu')(meta)
    concat = tf.keras.layers.concatenate([meta_output, output], name = "concat")
    
    output = tf.keras.layers.Dense(2,activation='softmax', name = "head")(concat)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks, meta],outputs = output)
    
    
    for layer in model.layers[:3]:
        print(layer)
        layer.trainable = False
    
    return model

In [14]:
model = build_model()

In [15]:
followers = X_meta.to_numpy().astype('float32')
X_train, X_test, y_train, y_test,train_mask,test_mask, meta_train, meta_test =train_test_split(input_ids,
                                                                       target,
                                                                       attention_masks,
                                                                       followers, test_size=0.3)
scaler = StandardScaler()
meta_train = scaler.fit_transform(meta_train)
meta_test = scaler.transform(meta_test)
del input_ids, attention_masks, X_meta
gc.collect()

50

In [16]:
trainloader = tf.data.Dataset.from_tensor_slices(({'input_ids':X_train, 'attention':train_mask, 'meta': meta_train}, y_train))
testloader = tf.data.Dataset.from_tensor_slices(({'input_ids':X_test, 'attention':test_mask, 'meta': meta_test}, y_test))
trainloader = (
        trainloader
        .cache()
        .shuffle(2048)
        .batch(batch_size)
        .prefetch(AUTOTUNE)
        
    )

testloader = (
        testloader
        .batch(512)
        .prefetch(AUTOTUNE)
    )

In [17]:
rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr= min_lr)
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
opt = tf.keras.optimizers.Adam(lr = start_lr)
checkpoint_filepath = '/content/drive/MyDrive/datasocio/checkpointfriends'
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq = 'epoch',
    monitor='val_loss',
    mode='max',
    save_best_only=True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [18]:
model.compile(loss='categorical_crossentropy',optimizer= opt,metrics=['categorical_accuracy', tf.keras.metrics.AUC()])
#model.optimizer = opt

In [19]:
model.load_weights('/content/drive/MyDrive/datasocio/withmeta_friends.h5')

In [ ]:
#leakage in metadata, to be fixed
history=model.fit(trainloader,
                        validation_data = testloader,
                        batch_size=batch_size,
                        callbacks = [rlr, es, ckpt],
                        epochs=epochs)

Epoch 1/10
12482/15296 [=======================>......] - ETA: 35:51 - loss: 0.5834 - categorical_accuracy: 0.6781 - auc: 0.7501

In [ ]:
#leakage in metadata, to be fixed
history=model.fit(trainloader,
                        validation_data = testloader,
                        batch_size=batch_size,
                        callbacks = [rlr, es, ckpt],
                        epochs=epochs)

In [ ]:
model.save_weights('/content/drive/MyDrive/datasocio/withmeta_friends.h5')